# Part 1

--- Day 2: Rock Paper Scissors ---

The Elves begin to set up camp on the beach. To decide whose tent gets to be closest to the snack storage, a giant Rock Paper Scissors tournament is already in progress.

Rock Paper Scissors is a game between two players. Each game contains many rounds; in each round, the players each simultaneously choose one of Rock, Paper, or Scissors using a hand shape. Then, a winner for that round is selected: Rock defeats Scissors, Scissors defeats Paper, and Paper defeats Rock. If both players choose the same shape, the round instead ends in a draw.

Appreciative of your help yesterday, one Elf gives you an encrypted strategy guide (your puzzle input) that they say will be sure to help you win. "The first column is what your opponent is going to play: A for Rock, B for Paper, and C for Scissors. The second column--" Suddenly, the Elf is called away to help with someone's tent.

The second column, you reason, must be what you should play in response: X for Rock, Y for Paper, and Z for Scissors. Winning every time would be suspicious, so the responses must have been carefully chosen.

The winner of the whole tournament is the player with the highest score. Your total score is the sum of your scores for each round. The score for a single round is the score for the shape you selected (1 for Rock, 2 for Paper, and 3 for Scissors) plus the score for the outcome of the round (0 if you lost, 3 if the round was a draw, and 6 if you won).

Since you can't be sure if the Elf is trying to help you or trick you, you should calculate the score you would get if you were to follow the strategy guide.

For example, suppose you were given the following strategy guide:

A Y

B X

C Z

This strategy guide predicts and recommends the following:

In the first round, your opponent will choose Rock (A), and you should choose Paper (Y). This ends in a win for you with a score of 8 (2 because you chose Paper + 6 because you won).
In the second round, your opponent will choose Paper (B), and you should choose Rock (X). This ends in a loss for you with a score of 1 (1 + 0).
The third round is a draw with both players choosing Scissors, giving you a score of 3 + 3 = 6.
In this example, if you were to follow the strategy guide, you would get a total score of 15 (8 + 1 + 6).

What would your total score be if everything goes exactly according to your strategy guide?

In [1]:
from enum import Enum
from functools import total_ordering

@total_ordering
class Shapes(Enum):
    ROCK = 1
    PAPER = 2
    SCISSORS = 3
    
    # Loss condition
    def __lt__(self, other):
        if self.__class__ is other.__class__:
            if ((self == Shapes.ROCK and other == Shapes.PAPER)
                or (self == Shapes.PAPER and other == Shapes.SCISSORS)
                or (self == Shapes.SCISSORS and other == Shapes.ROCK)):
                return True
            else:
                return False
        return NotImplemented

class Score(Enum):
    WIN = 6
    DRAW = 3
    LOSE = 0
    
# Unify the notation
rps_map = {'A': Shapes.ROCK, 'B': Shapes.PAPER, 'C': Shapes.SCISSORS,
           'X': Shapes.ROCK, 'Y': Shapes.PAPER, 'Z': Shapes.SCISSORS}

with open('input.txt', 'r') as f:
    score = 0
    for l in f:
        opp, you = [rps_map[s] for s in l.split()]
        if opp == you:
            score += Score.DRAW.value
        elif opp > you:
            score += Score.LOSE.value
        else:
            score += Score.WIN.value
        score += you.value
print(f'Expected total score is {score}.')

Expected total score is 12586.


# Part 2

The Elf finishes helping with the tent and sneaks back over to you. "Anyway, the second column says how the round needs to end: X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win. Good luck!"

The total score is still calculated in the same way, but now you need to figure out what shape to choose so the round ends as indicated. The example above now goes like this:

In the first round, your opponent will choose Rock (A), and you need the round to end in a draw (Y), so you also choose Rock. This gives you a score of 1 + 3 = 4.
In the second round, your opponent will choose Paper (B), and you choose Rock so you lose (X) with a score of 1 + 0 = 1.
In the third round, you will defeat your opponent's Scissors with Rock for a score of 1 + 6 = 7.
Now that you're correctly decrypting the ultra top secret strategy guide, you would get a total score of 12.

Following the Elf's instructions for the second column, what would your total score be if everything goes exactly according to your strategy guide?

In [2]:
@total_ordering
class Shapes(Enum):
    ROCK = 1
    PAPER = 2
    SCISSORS = 3
    
    # Loss condition
    def __lt__(self, other):
        if self.__class__ is other.__class__:
            # If the other shape wins vs self
            if Shapes.permute(self) == other:
                return True
            else:
                return False
        return NotImplemented
    
    # Transform to the winning shape
    def permute_r(self):
        return Shapes(self.value % 3 + 1)
    
    # Transform to the losing shape
    def permute_l(self):
        return Shapes.permute_r(Shapes.permute_r(self))

# Unify the notation
rps_map = {'A': Shapes.ROCK, 'B': Shapes.PAPER, 'C': Shapes.SCISSORS,
           'X': Score.LOSE, 'Y': Score.DRAW, 'Z': Score.WIN}

with open('input.txt', 'r') as f:
    score = 0
    for l in f:
        opp, result = [rps_map[s] for s in l.split()]
        if result == Score.DRAW:
            score += opp.value
        elif result == Score.WIN:
            score += Shapes.permute_r(opp).value
        else:
            score += Shapes.permute_l(opp).value
        score += result.value
print(f'Expected total score is {score}.')

Expected total score is 13193.
